In [6]:
from shapely.geometry import MultiPoint, LineString, MultiLineString, Point
from shapely.geometry import polygon 
from shapely.ops import nearest_points, linemerge
import numpy as np
# from libpysal.cg.voronoi import voronoi, voronoi_frames
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.ops import unary_union, polygonize, cascaded_union, split, polygonize_full, snap
from shapely import affinity
from datetime import datetime
from os.path import exists
import pygeos as pg
import os
import math

In [7]:
df_qv = gpd.read_file('gis/SIRGAS_GPKG_quadraviariaed_2017.gpkg')
df_l = gpd.read_file(f'zip://gis/SIRGAS_SHP_logradouronbl.zip!SIRGAS_SHP_logradouronbl/SIRGAS_SHP_logradouronbl.shp')

In [8]:
df_dist = gpd.read_file(f'zip://gis/SIRGAS_SHP_distrito.zip!SIRGAS_SHP_distrito/SIRGAS_SHP_distrito_polygon.shp')
df_represas = gpd.read_file(f'zip://gis/SIRGAS_SHP_represanivelmax.zip!SIRGAS_SHP_represanivelmax').set_crs(epsg=31983)
df_massa_dagua = gpd.read_file(f'zip://gis/SIRGAS_SHP_hidrogramassadagua.zip!SIRGAS_SHP_hidromassadagua/SIRGAS_SHP_hidromassadagua_polygon.shp').set_crs(epsg=31983)

In [9]:
df_cruzamentos = gpd.read_file(f'resultado/pre-processamentos/00_cruzamentos_cidade_sao_paulo.gpkg')

In [5]:
# gpd.GeoDataFrame(geometry=qv.buffer(0.001)).dissolve().explode(index_parts=False).reset_index().drop(['index'], axis=1)

In [12]:
for id, dist in df_dist.iterrows():

    print(f'Processando meio-fio de {dist.ds_nome}')
    if exists(f'resultado/pre-processamentos/{dist.ds_codigo}_meio-fio_{dist.ds_nome}.gpkg'):
        continue

    # qv = df_qv.loc[df_qv.qe_tipo == 'Quadra'].clip(dist.geometry)
    qv = df_qv.clip(dist.geometry)
    qv = gpd.GeoDataFrame(geometry=qv.buffer(0.001)).dissolve().explode(index_parts=False).reset_index().drop(['index'], axis=1)

    l = df_l.clip(dist.geometry.buffer(20))

    cruzamentos = df_cruzamentos.clip(dist.geometry.buffer(20))

    cruzamentos_buffer = gpd.GeoDataFrame(geometry=cruzamentos.buffer(35)).set_crs(epsg=31983)

    esquinas = qv.overlay(cruzamentos_buffer.dissolve(), how='intersection').explode(index_parts=False).reset_index()

    pontos_corte = \
    [nearest_points(cruzamentos.loc[i].geometry, esquinas.loc[r.index_right, 'geometry'])[-1] 
    for i, r in cruzamentos_buffer.sjoin(esquinas, how='left', predicate='intersects').iterrows() 
    if not math.isnan(r.index_right)]

    qv_boundary = gpd.GeoDataFrame(geometry=[polygon.orient(g).boundary for g in qv.explode().geometry]).unary_union

    linhas_preparadas = split(qv_boundary, snap(MultiPoint(pontos_corte), qv_boundary, 0.10).buffer(0.001))

    geometrias = [i for i in linhas_preparadas]

    meios_fios = gpd.GeoDataFrame(geometry=geometrias)

    meio_fio_ponto_medio = gpd.GeoDataFrame(meios_fios.copy(), geometry=[l.interpolate(0.5, normalized=True) for l in meios_fios.geometry]).sjoin_nearest(l, how='left').drop_duplicates()

    meios_fios.loc[meio_fio_ponto_medio['lg_codlog'].index, 'lg_codlog'] = meio_fio_ponto_medio['lg_codlog']
    meios_fios.loc[meio_fio_ponto_medio['lg_seg_id'].index, 'lg_seg_id'] = meio_fio_ponto_medio['lg_seg_id']
    meios_fios.loc[meio_fio_ponto_medio.index, 'lado_esquerdo'] = [r.geometry.intersects(l.loc[r.index_right].geometry.buffer(35, single_sided=True)) for i, r in meio_fio_ponto_medio.iterrows()]
    meios_fios.lg_codlog = meios_fios.lg_codlog.fillna(0.)

    meios_fios = meios_fios.dissolve(by=['lg_codlog', 'lg_seg_id', 'lado_esquerdo']).reset_index()

    meios_fios.to_file(f'resultado/pre-processamentos/{dist.ds_codigo}_meio-fio_{dist.ds_nome}.gpkg', 
                        driver='GPKG', crs=31983)

    # break

Processando meio-fio de MOEMA


/tmp/ipykernel_155189/152124283.py:24: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  qv_boundary = gpd.GeoDataFrame(geometry=[polygon.orient(g).boundary for g in qv.explode().geometry]).unary_union
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Processando meio-fio de PARQUE DO CARMO


/tmp/ipykernel_155189/152124283.py:24: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  qv_boundary = gpd.GeoDataFrame(geometry=[polygon.orient(g).boundary for g in qv.explode().geometry]).unary_union
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Processando meio-fio de PERDIZES


...

/tmp/ipykernel_155189/152124283.py:24: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  qv_boundary = gpd.GeoDataFrame(geometry=[polygon.orient(g).boundary for g in qv.explode().geometry]).unary_union
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Processando meio-fio de JARDIM HELENA


/tmp/ipykernel_155189/152124283.py:24: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  qv_boundary = gpd.GeoDataFrame(geometry=[polygon.orient(g).boundary for g in qv.explode().geometry]).unary_union
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Processando meio-fio de JARDIM PAULISTA


/tmp/ipykernel_155189/152124283.py:24: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  qv_boundary = gpd.GeoDataFrame(geometry=[polygon.orient(g).boundary for g in qv.explode().geometry]).unary_union
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
